# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
import pickle
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from sqlalchemy import create_engine

%matplotlib inline

* Load data from database

In [3]:
engine = create_engine('sqlite:///disaster_response_db.db')
df = pd.read_sql_table('disaster_response_db_table',engine)  

In [4]:
pd.set_option('display.max_columns', 500)
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.0,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.00000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000
mean,15142.682375,0.774526,0.170947,0.004541,0.415080,0.079635,0.050139,0.027626,0.017972,0.032816,0.0,0.063876,0.111802,0.088488,0.015492,0.023047,0.011409,0.033426,0.045637,0.131568,0.065059,0.045904,0.050941,0.020376,0.006067,0.010799,0.004579,0.011791,0.04392,0.278628,0.082344,0.093410,0.010760,0.093601,0.020224,0.052505,0.193841
std,8780.194298,0.435081,0.376470,0.067233,0.492745,0.270733,0.218236,0.163902,0.132853,0.178157,0.0,0.244537,0.315129,0.284008,0.123502,0.150057,0.106205,0.179750,0.208700,0.338027,0.246635,0.209280,0.219881,0.141286,0.077656,0.103356,0.067514,0.107945,0.20492,0.448333,0.274894,0.291012,0.103175,0.291278,0.140767,0.223047,0.395314
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7408.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15568.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22805.500000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30111.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


* Remove child_alone because its column has all zeros.

In [5]:
df = df.drop(['child_alone'],axis=1)

* Extract X and y variables from the dataset

In [6]:
X = df["message"]
Y = df[df.columns[4:]]

In [7]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [8]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 2. Pre-process Text Data 
* Write a tokenization function to process the text data

In [9]:
def tokenize(text,url_place_holder_string="urlplaceholder"):
    
    
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

* Build a custom transformer which will extract the starting verb of a sentence

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('moc', MultiOutputClassifier(RandomForestClassifier(random_state=1)))
])

In [11]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f5c570b61e0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('moc',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=1, ve

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline_fitted = pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
Y_prediction_train = pipeline_fitted.predict(X_train)
Y_prediction_test = pipeline_fitted.predict(X_test)

In [14]:
category_names = Y.columns
for i in range(len(category_names)):
    print("Category:", category_names[i],"\n", classification_report(Y_test.iloc[:, i].values, Y_prediction_test[:, i]))
    print('Accuracy of %25s: %.2f' %(category_names[i], accuracy_score(Y_test.iloc[:, i].values, Y_prediction_test[:,i])))

Category: related 
              precision    recall  f1-score   support

          0       0.28      0.11      0.15      1486
          1       0.77      0.92      0.84      5024
          2       0.00      0.00      0.00        42

avg / total       0.65      0.73      0.68      6552

Accuracy of                   related: 0.73
Category: request 
              precision    recall  f1-score   support

          0       0.83      0.98      0.90      5428
          1       0.38      0.06      0.11      1124

avg / total       0.76      0.82      0.77      6552

Accuracy of                   request: 0.82
Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6552

Accuracy of                     offer: 1.00
Category: aid_related 
              precision    recall  f1-score   support

          0       0.59      0.83  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters = {
    'moc__estimator__criterion': ['gini', 'entropy'],
    'moc__estimator__n_estimators': [5, 10, 20],
    'moc__estimator__min_samples_split': [2, 4, 6]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, n_jobs=-1)
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] moc__estimator__criterion=gini, moc__estimator__min_samples_split=2, moc__estimator__n_estimators=5 
[CV]  moc__estimator__criterion=gini, moc__estimator__min_samples_split=2, moc__estimator__n_estimators=5, score=0.09829059829059829, total=  34.3s
[CV] moc__estimator__criterion=gini, moc__estimator__min_samples_split=2, moc__estimator__n_estimators=5 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   43.1s remaining:    0.0s


[CV]  moc__estimator__criterion=gini, moc__estimator__min_samples_split=2, moc__estimator__n_estimators=5, score=0.09829059829059829, total=  34.5s
[CV] moc__estimator__criterion=gini, moc__estimator__min_samples_split=2, moc__estimator__n_estimators=5 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  1.4min remaining:    0.0s


[CV]  moc__estimator__criterion=gini, moc__estimator__min_samples_split=2, moc__estimator__n_estimators=5, score=0.09937414135246528, total=  35.0s
[CV] moc__estimator__criterion=gini, moc__estimator__min_samples_split=2, moc__estimator__n_estimators=10 
[CV]  moc__estimator__criterion=gini, moc__estimator__min_samples_split=2, moc__estimator__n_estimators=10, score=0.15491452991452992, total=  59.6s
[CV] moc__estimator__criterion=gini, moc__estimator__min_samples_split=2, moc__estimator__n_estimators=10 
[CV]  moc__estimator__criterion=gini, moc__estimator__min_samples_split=2, moc__estimator__n_estimators=10, score=0.14957264957264957, total=  58.9s
[CV] moc__estimator__criterion=gini, moc__estimator__min_samples_split=2, moc__estimator__n_estimators=10 
[CV]  moc__estimator__criterion=gini, moc__estimator__min_samples_split=2, moc__estimator__n_estimators=10, score=0.15280109906884445, total=  59.8s
[CV] moc__estimator__criterion=gini, moc__estimator__min_samples_split=2, moc__estim

[CV]  moc__estimator__criterion=entropy, moc__estimator__min_samples_split=2, moc__estimator__n_estimators=20, score=0.15934065934065933, total= 1.7min
[CV] moc__estimator__criterion=entropy, moc__estimator__min_samples_split=2, moc__estimator__n_estimators=20 
[CV]  moc__estimator__criterion=entropy, moc__estimator__min_samples_split=2, moc__estimator__n_estimators=20, score=0.15875438864295527, total= 1.7min
[CV] moc__estimator__criterion=entropy, moc__estimator__min_samples_split=4, moc__estimator__n_estimators=5 
[CV]  moc__estimator__criterion=entropy, moc__estimator__min_samples_split=4, moc__estimator__n_estimators=5, score=0.1130952380952381, total=  28.3s
[CV] moc__estimator__criterion=entropy, moc__estimator__min_samples_split=4, moc__estimator__n_estimators=5 
[CV]  moc__estimator__criterion=entropy, moc__estimator__min_samples_split=4, moc__estimator__n_estimators=5, score=0.11706349206349206, total=  28.8s
[CV] moc__estimator__criterion=entropy, moc__estimator__min_samples

[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed: 61.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...1,
            oob_score=False, random_state=1, verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'moc__estimator__criterion': ['gini', 'entropy'], 'moc__estimator__n_estimators': [5, 10, 20], 'moc__estimator__min_samples_split': [2, 4, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [19]:
cv.best_params_

{'moc__estimator__criterion': 'gini',
 'moc__estimator__min_samples_split': 2,
 'moc__estimator__n_estimators': 20}

In [18]:
# Get the prediction values from the grid search cross validator
Y_prediction_test = cv.predict(X_test)
Y_prediction_train = cv.predict(X_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
for i in range(len(category_names)):
    print("Category:", category_names[i],"\n", classification_report(Y_train.iloc[:, i].values, Y_prediction_train[:, i]))
    print('Accuracy of %25s: %.2f' %(category_names[i], accuracy_score(Y_train.iloc[:, i].values, Y_prediction_train[:,i])))

Category: related 
              precision    recall  f1-score   support

          0       1.00      0.98      0.99      4615
          1       0.99      1.00      1.00     14890
          2       0.99      0.89      0.94       150

avg / total       0.99      0.99      0.99     19655

Accuracy of                   related: 0.99
Category: request 
              precision    recall  f1-score   support

          0       0.99      1.00      0.99     16299
          1       1.00      0.95      0.97      3356

avg / total       0.99      0.99      0.99     19655

Accuracy of                   request: 0.99
Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00     19562
          1       1.00      0.84      0.91        93

avg / total       1.00      1.00      1.00     19655

Accuracy of                     offer: 1.00
Category: aid_related 
              precision    recall  f1-score   support

          0       0.99      1.00  

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [22]:
m = pickle.dumps('MLpipeline.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.